In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# https://huggingface.co/Qwen/Qwen3-0.6B
model_name = "Qwen/Qwen3-0.6B"

c:\Users\mirxm\Storage\Work\MDS\S3\MLOps\CriteriaEvaluator\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mirxm\.cache\huggingface\hub\models--gpt2-xl. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to 

In [3]:
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")

`torch_dtype` is deprecated! Use `dtype` instead!


In [4]:
# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [{"role": "user", "content": prompt}]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,  # Switches between thinking and non-thinking modes. Default is True.
)
print("Model input:")
print(text)

Model input:
<|im_start|>user
Give me a short introduction to large language model.<|im_end|>
<|im_start|>assistant
<think>

</think>




In [5]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
print("Tokenized input:")
print(model_inputs)

Tokenized input:
{'input_ids': tensor([[151644,    872,    198,  35127,    752,    264,   2805,  16800,    311,
           3460,   4128,   1614,     13, 151645,    198, 151644,  77091,    198,
         151667,    271, 151668,    271]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}


In [6]:
# conduct text completion
generated_ids = model.generate(**model_inputs, max_new_tokens=32768)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]) :].tolist()

In [7]:
# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("Thinking content:\n", thinking_content)
print("Content:\n", content)

Thinking content:
 
Content:
 A large language model (LLM) is a type of artificial intelligence model that can understand, generate, and respond to natural language tasks. It's trained on vast datasets to learn patterns and understand context, allowing it to answer questions, write texts, and perform various tasks.


Try a single model execution to visualize the computation graph.

In [8]:
out = model(**model_inputs).logits
print(out)

tensor([[[ 3.6094,  3.7969,  3.6562,  ...,  1.6484,  1.6484,  1.6484],
         [ 7.0312,  8.3125,  6.4062,  ...,  0.4492,  0.4492,  0.4492],
         [ 4.7188,  9.6250, 11.5625,  ...,  3.8594,  3.8594,  3.8594],
         ...,
         [ 9.1875, 14.6250,  9.9375,  ...,  0.4707,  0.4707,  0.4707],
         [-4.5625, -3.3906, -8.1250,  ..., -1.2734, -1.2734, -1.2734],
         [11.5000, 15.8125,  9.8125,  ...,  3.6875,  3.6875,  3.6875]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>)


In [9]:
from torchviz import make_dot


make_dot(out, params=dict(model.named_parameters()))